In [1]:
# !pip install pytube

In [2]:
# importing DS module
import os
import json
import glob
import joblib
import numpy as np
import pandas as pd

# importing data processing module  
import cv2
from pytube import YouTube  


from utils.VideoLoader import VideoLoader
from utils.VideoFrameDataset import VideoFrameDataset

from tqdm.notebook import tqdm

In [3]:
# read path
DATA_DIR = '../data'

# where to save  
RAW_VIDEO_PATH = "../data/raw/video/train/"
PREPROCESS_VIDEO_PATH = "../data/preprocessed/video/train/"

In [4]:
# read the json file
with open(DATA_DIR + '/metadata/MS-ASL/MSASL_train.json') as f:
  train_data = json.load(f)

# turn json to np array
train_data = pd.DataFrame(train_data)
train_data.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360.0,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360.0,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360.0,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN


In [5]:
train_data['box'].iloc[0]

[0.05754461884498596, 0.21637457609176636, 1.0, 0.7300844192504883]

In [5]:
train_data.to_csv(DATA_DIR + '/metadata/MS-ASL/MSASL_train_metadata.csv', index=False)

# Preprocess MetaData
1. Remove Invalid Data
2. Rename Video ID 

In [15]:
# if the video exits append it to a datafranme
# create a dataframe to store the video metadata
valid_metadata = pd.DataFrame(columns=['name', 'start', 'end', 'clean_text'])
for i, row in tqdm(train_data.iterrows()):
    row['name'] = row['url'].split('=')[1]
    video_path = f'{RAW_VIDEO_PATH}/{row["name"]}.mp4'
    if os.path.exists(video_path):
        # print(pd.DataFrame(row[['SENTENCE_NAME', 'START_REALIGNED', 'END_REALIGNED', 'SENTENCE']]).T)
        valid_metadata = pd.concat([valid_metadata, pd.DataFrame(row[['name', 'start', 'end', 'clean_text']]).T], ignore_index=True, axis=0)
valid_metadata.to_csv(f'{DATA_DIR}/metadata/MS-ASL/valid_MSASL_train_metadata.csv', index=False)
valid_metadata.head(), len(valid_metadata)

0it [00:00, ?it/s]

(          name start  end     clean_text
 0  C37R_Ix8-qs     0   83          match
 1  PIsUJl8BN_I     0   74           fail
 2  J7tP98oDxqE     0   66           book
 3  N2mG9ZKjrGA     0   75  sign language
 4  SVWABYmFdhs     0  116         easter,
 12460)

# Convert Video to Numpy
Download the video and convert cette video to numpy array with video id, video array, and clean text label

In [6]:
# for i, row in tqdm(train_data.iterrows()):
#     # wait 20 seconds
#     # time.sleep(20)

#     # link of the video to be downloaded  
#     link = row['url']
#     id = link.split('=')[1]

#     # Try downloading the video
#     try:  
#         # object creation using YouTube 
#         # which was imported in the beginning  
#         yt = YouTube(link)  
#         # downloading the video  
#         yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution')[-1].download(RAW_VIDEO_PATH, filename=id+".mp4") 

#     except Exception as e:  
#         print(f"Error: {e}")  




# Experimental Code

In [7]:
# # Load videos using VideoFrameDataset
# train_data['name'] = train_data['url'].apply(lambda x: x.split('=')[1])
# metadata = train_data[['name', 'start', 'end', 'clean_text']].values


# # Load videos using VideoFrameDataset
# video_dataset = VideoFrameDataset(
#     root_path=RAW_VIDEO_PATH,
#     metadata=metadata,
#     num_segments=5,
#     frames_per_segment=1,
#     transform=None,
#     test_mode=False
# )

# for i, sample in enumerate(video_dataset):
#     print(i, sample[0].shape, sample[1])
#     if i == 10: break

In [16]:
# Load videos using VideoFrameDataset
# train_data['name'] = train_data['url'].apply(lambda x: x.split('=')[1])
metadata = valid_metadata[['name', 'start', 'end', 'clean_text']].values

video_dataset = VideoLoader(root_path = RAW_VIDEO_PATH,
                            metadata = metadata)

for i, sample in enumerate(video_dataset):
    print(i, sample['video'].shape, sample['label'])
    if i == 10: break

0 torch.Size([85, 3, 720, 1280]) match
1 torch.Size([76, 3, 360, 480]) fail
2 torch.Size([68, 3, 360, 480]) book
3 torch.Size([77, 3, 720, 1280]) sign language
4 torch.Size([118, 3, 720, 1280]) easter
5 torch.Size([72, 3, 720, 1280]) boring
6 torch.Size([205, 3, 720, 1280]) phone
7 torch.Size([205, 3, 720, 1280]) phone
8 torch.Size([205, 3, 720, 1280]) phone
9 torch.Size([75, 3, 360, 480]) library
10 torch.Size([115, 3, 720, 1280]) cochlear implant


In [8]:
# video_arrays = []

# for i, row in tqdm(train_data.iterrows()):
#     # wait 20 seconds
#     # time.sleep(20)

#     # link of the video to be downloaded  
#     link = row['url']
#     id = link.split('=')[1]
#     label = row['clean_text']


#     # Initialize a VideoCapture object to read from the video file
#     video_path = RAW_VIDEO_PATH+f'/{id}.mp4'
#     cap = cv2.VideoCapture(video_path)


#     # Check if the video was opened successfully
#     if not cap.isOpened():
#         print("Error: Could not open video.")
#         continue


#     # Read the frames from the video
#     frames = []
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break  # Break the loop if there are no frames left to read

#         # Append the frame to the list of frames
#         frames.append(frame)

#     # Convert the list of frames to a NumPy array
#     video_array = np.array(frames)

#     # Release the VideoCapture object
#     cap.release()

#     # append the video array to the list of video arrays
#     data = (id, video_array, label)
#     video_arrays.append(data)


# # save data as pickle file
# joblib.dump(video_arrays, f'{PREPROCESS_VIDEO_PATH}/train.pkl')